In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("job.csv", encoding='latin1')
df

,Company,Job Title,Location,Job Type,Experience level,Salary,Requirment of the company,Facilities
0,SGS,Clinical Data Analyst,"Richardson, TX, United States",Full Time,Entry-level,48K+ *,"Computer Science,Data quality,Genetics,Mathema...",",,,,"
1,Ocorian,AML/CFT & Data Analyst,"Ebène, Mauritius",Full Time,Entry-level,48K+ *,"Agile,Data management,Finance,Security,,",",,,,"
2,Cricut,Machine Learning Engineer,"South Jordan, UT, United States",Full Time,NaN,90K+ *,"Agile,Architecture,AWS,Computer Science,Comput...","Career development,,,,"
3,Bosch Group,Application Developer & Data Analyst,"Nonantola, Italy",Full Time,Entry-level,48K+ *,"Engineering,Industrial,Oracle,Power BI,R,R&D",",,,,"
4,Publicis Groupe,Data Engineer Full time (Public Sector) USA,"Arlington, VA, United States",Full Time,Mid-level,108K+,"AWS,Azure,Computer Science,Consulting,Dataflow...","Flex hours,Flex vacation,Parental leave,Unlimi..."
...,...,...,...,...,...,...,...,...
3193,Western Digital,Data Scientist - New College Graduate,"Biñan, Philippines",Full Time,Entry-level,39K+ *,"APIs,Clustering,Computer Science,Data visualiz...","Career development,,,,"
3194,Experian,Cloud Data Analyst,"Heredia, Costa Rica",Full Time,Senior-level,92K+ *,"AWS,Big Data,Computer Science,GCP,Snowflake,SQL","Equity,Medical leave,Salary bonus,,"
3195,Locus Robotics,"Robotics Engineer, Sensors","Wilmington, MA, United States",Full Time,Senior-level,62K+ *,"E-commerce,Engineering,Linux,Python,Robotics,S...",",,,,"
3196,ATB Financial,Data Scientist,"Edmonton, Alberta, Canada",Full Time,Entry-level,39K+ *,"Computer Science,Data Analytics,Data Mining,Ec...","Career development,Startup environment,,,"


In [4]:
import pyodbc


In [ ]:

# Connexion à la base de données SQL Server
conn = pyodbc.connect("Driver={SQL Server};Server=DESKTOP-MLS7M99\SQLEXPRESS;Database=emploi;Trusted_Connection=yes;")
cursor = conn.cursor()

cursor.execute("SELECT @@version;")
row = cursor.fetchone()
while row: 
    print(row[0])
    row = cursor.fetchone()


In [ ]:
#create table job 
job_table = df[["Job Title", "Salary", "Plus", "Asterisk", "Experience level"]].reset_index(drop=True)
job_table.insert(0, "id_job", range(1, len(job_table) + 1))
display(job_table)

In [ ]:
# Définir la fonction pour insérer les données de la table "Job" dans la base de données
def insert_job_data(cursor, job_data):
    for index, row in job_data.iterrows():
        cursor.execute("INSERT INTO Job (id_job, job_title, salary, plus, Asterisk, level_exp) VALUES (?, ?, ?, ?, ?, ?)",
                       row["id_job"], row["Job Title"], row["Salary"], row["Plus"], row["Asterisk"], row["Experience level"])
    conn.commit()

# Appel de la fonction pour insérer les données dans la table "Job"
insert_job_data(cursor, job_table)

# Fermeture de la connexion à la base de données


In [ ]:
# Insertion des données dans la table "Company"
company_table = df[["Company"]].drop_duplicates().reset_index(drop=True)

company_table.insert(0, "id_company", range(1, len(company_table) + 1))
display(company_table)

In [ ]:
# Insertion des données de la table "Company" dans la base de donnees

def insert_company_data(cursor, company_data):
    for index, row in company_data.iterrows():
        cursor.execute("INSERT INTO Company (id_company, company_name) VALUES (?, ?)", row["id_company"], row["Company"])
    conn.commit()

    # Appel de la fonction pour insérer les données dans la table "Company"
insert_company_data(cursor, company_table)

# Fermeture de la connexion à la base de données
conn.close()

In [ ]:
#table Job_Company
df_jc = df.copy()

job_company = df_jc.merge(company_table, on="Company" ).merge(job_table, on="Job Title")
job_company = job_company[["id_job", "id_company"]]
job_company


In [ ]:
job_company.drop_duplicates()

In [ ]:
# Insertion des données de la table "Job_Company" dans la base de donnees

def insert_job_company_data(cursor, job_company_data):
    for index, row in job_company_data.iterrows():
        id_job = int(row["id_job"])
        id_company = int(row["id_company"])
        cursor.execute("INSERT INTO Job_Company (id_job, id_company) VALUES (?, ?)", (id_job, id_company))
    conn.commit()

# Appel de la fonction pour insérer les données dans la table "job_company"
insert_job_company_data(cursor, job_company)

In [ ]:
# Insertion des données dans la table "country_table"
country_table = df[["Country"]].drop_duplicates().reset_index(drop=True)

country_table.insert(0, "id_country", range(1, len(country_table) + 1))
display(country_table)

In [ ]:
# Insertion des données de la table "Country" dans la base de donnees

def insert_country_data(cursor, country_data):
    for index, row in country_data.iterrows():
        cursor.execute("INSERT INTO Country (id_country, country) VALUES (?, ?)", row["id_country"], row["Country"])
    conn.commit()

    # Appel de la fonction pour insérer les données dans la table "Country"
insert_country_data(cursor, country_table)

# Fermeture de la connexion à la base de données


In [ ]:
#table Job_Country  
df_jct = df.copy()

job_country = df_jct.merge(country_table, on="Country" ).merge(job_table, on="Job Title")
job_country = job_country[["id_job", "id_country"]]
job_country


In [ ]:
# Insertion des données de la table "Job_Country" dans la base de donnees

def insert_job_country_data(cursor, job_country_data):
    for index, row in job_country_data.iterrows():
        id_job = int(row["id_job"])
        id_country = int(row["id_country"])
        cursor.execute("INSERT INTO Job_Country (id_job, id_country) VALUES (?, ?)", (id_job, id_country))
    conn.commit()

# Appel de la fonction pour insérer les données dans la table "Job_Country"
insert_job_country_data(cursor, job_country)

In [ ]:
# Sélectionner la colonne "Facilities" et supprimer les doublons
Facilities_table = df_copy1[["Facilities"]].drop_duplicates().reset_index(drop=True)

# Insérer une colonne "id_facilities" avec des valeurs croissantes à partir de 1
Facilities_table.insert(0, "id_facilities", range(1, len(Facilities_table) + 1))

# Afficher la table "Facilities"
display(Facilities_table)

In [ ]:
# Insertion des données de la table "Facilities" dans la base de donnees

def insert_facilities_data(cursor, facilities_data):
    for index, row in facilities_data.iterrows():
        cursor.execute("INSERT INTO Facilities (id_facilities, facilities) VALUES (?, ?)", row["id_facilities"], row["Facilities"])
    conn.commit()

# Appel de la fonction pour insérer les données dans la table "Facilities"
insert_facilities_data(cursor, Facilities_table)

# Fermeture de la connexion à la base de données

In [ ]:
# Maintenant vous pouvez effectuer le merge avec la table "Facilities_table" et "job_table"
job_Facilities = df_copy1.merge(Facilities_table, on="Facilities").merge(job_table, on="Job Title")
job_Facilities = job_Facilities[["id_job", "id_facilities"]]
job_Facilities


In [ ]:
# Insertion des données de la table "Job_Facilities" dans la base de donnees

def insert_job_facilities_data(cursor, job_facilities_data):
    for index, row in job_facilities_data.iterrows():
        id_job = int(row["id_job"])
        id_facilities = int(row["id_facilities"])
        cursor.execute("INSERT INTO Job_Facilities (id_job, id_facilities) VALUES (?, ?)", (id_job, id_facilities))
    conn.commit()
    
        
insert_job_facilities_data(cursor, job_Facilities)

# Fermeture de la connexion à la base de données


In [ ]:
# Insertion des données dans la table "Requirment"
Requirment_table = df_copy[["Requirment of the company"]].drop_duplicates().reset_index(drop=True)

Requirment_table.insert(0, "id_requirment", range(1, len(Requirment_table) + 1))
display(Requirment_table)



In [ ]:
# Insertion des données de la table "Requirment" dans la base de donnees

def insert_requirment_data(cursor, requirment_data):
    for index, row in requirment_data.iterrows():
        cursor.execute("INSERT INTO Requirements (id_requirement, requirement_company) VALUES (?, ?)", row["id_requirment"], row["Requirment of the company"])
    conn.commit()

    # Appel de la fonction pour insérer les données dans la table "Company"
insert_requirment_data(cursor, Requirment_table)

# Fermeture de la connexion à la base de données


In [ ]:
# df_copy
job_requirements = df_copy.merge(Requirment_table, on="Requirment of the company" ).merge(job_table, on="Job Title")
job_requirements = job_requirements[["id_job", "id_requirment"]]
job_requirements


In [ ]:
# Insertion des données de la table "Job_Requirements" dans la base de donnees

def insert_job_requirements_data(cursor, job_requirements_data):
    for index, row in job_requirements_data.iterrows():
        id_job = int(row["id_job"])
        id_requirment = int(row["id_requirment"])
        cursor.execute("INSERT INTO Job_Requirements (id_job, id_requirement) VALUES (?, ?)", (id_job, id_requirment))
    conn.commit()
    
insert_job_requirements_data(cursor, job_requirements)

# Fermeture de la connexion à la base de données
